
# Bitcoin News Sentiment Analysis API Demo

This notebook demonstrates how to use the Bitcoin News Sentiment Analysis API to fetch news articles, analyze their sentiment, and store the results.



## 1. Setup Environment

First, let's load environment variables and import the required modules.


In [54]:

# Import necessary libraries
import pandas as pd
from dotenv import load_dotenv
import os
from pathlib import Path

# Import our utility functions
from bitcoin_utils import (
    Config, fetch_bitcoin_news, classify_sentiment,
    process_news_articles, make_clickable, create_sentiment_pie_chart
)

# Load environment variables from .env file
load_dotenv()

# Check that we have the required API key
if not os.getenv("NEWS_API_KEY"):
    raise ValueError("NEWS_API_KEY not found in .env file")
    
print("Environment loaded successfully")


Environment loaded successfully



## 2. Initialize Configuration

Create a configuration object that will hold all our settings and API keys.


In [55]:

# Initialize the configuration
config = Config.load()

# Create data directory if it doesn't exist
config.DATA_DIR.mkdir(exist_ok=True)

print(f"Using LLM provider: {config.PROVIDER}")
print(f"Data directory: {config.DATA_DIR}")


Using LLM provider: together_ai
Data directory: /Users/sandeep/Desktop/Project25/Data605/data



## 3. Fetch Bitcoin News

Now let's fetch some recent Bitcoin news articles.


In [56]:

# Fetch latest Bitcoin news (last 3 days, 5 articles)
articles = fetch_bitcoin_news(config, page_size=30, days_back=30)

# Convert to DataFrame for easier viewing
df_articles = pd.DataFrame([
    {
        'title': article.get('title'),
        'publishedAt': article.get('publishedAt'),
        'source': article.get('source', {}).get('name'),
        'url': article.get('url')
    } for article in articles
])

# Display the articles
df_articles.head()


Fetching Bitcoin news from 2025-04-17 to 2025-05-17...


,title,publishedAt,source,url
0,Refer-a-Friend and You BOTH get a $20 BTC Bonu...,2025-05-16T08:10:15Z,Ozbargain.com.au,https://www.ozbargain.com.au/node/906089
1,Cetera Investment Advisers Decreases Holdings ...,2025-05-16T08:07:00Z,ETF Daily News,https://www.etfdailynews.com/2025/05/16/cetera...
2,"Mercer Global Advisors Inc. ADV Takes $312,000...",2025-05-16T08:00:53Z,ETF Daily News,https://www.etfdailynews.com/2025/05/16/mercer...
3,Bitcoin Could See Short-Term Holder Selloff Ar...,2025-05-16T08:00:49Z,newsBTC,http://www.newsbtc.com/bitcoin-news/bitcoin-sh...
4,Monero: Will an 82% rally in 40 days help XMR ...,2025-05-16T08:00:28Z,Ambcrypto.com,https://ambcrypto.com/monero-will-an-82-rally-...



## 4. Analyze Sentiment on Individual Headlines

Let's analyze the sentiment of a few headlines to demonstrate the API.


In [57]:

# Take the first 3 articles for demonstration
demo_articles = articles[:3]

# Analyze sentiment for each article
results = []
for article in demo_articles:
    title = article.get('title')
    content = article.get('description') or article.get('content', '')
    
    # Skip if no content
    if not content:
        print(f"No content for: {title}")
        continue
    
    # Analyze sentiment
    sentiment, cost = classify_sentiment(content, config)
    
    # Store result
    results.append({
        'headline': title,
        'content_preview': content[:100] + '...',
        'sentiment': sentiment,
        'score': 1 if sentiment.lower() == 'positive' else (-1 if sentiment.lower() == 'negative' else 0)
    })
    print(f"[{sentiment}] {title}")

# Display results
pd.DataFrame(results)


[Pos] Refer-a-Friend and You BOTH get a $20 BTC Bonus (Plus BOTH get $5 in Reward Points) @ Coinjar
[Ne] Cetera Investment Advisers Decreases Holdings in MARA Holdings, Inc. (NASDAQ:MARA)
[Ne] Mercer Global Advisors Inc. ADV Takes $312,000 Position in Bitwise Bitcoin ETF (NYSEARCA:BITB)


,headline,content_preview,sentiment,score
0,Refer-a-Friend and You BOTH get a $20 BTC Bonu...,"To celebrate our recent awards, we are spinnin...",Pos,0
1,Cetera Investment Advisers Decreases Holdings ...,Cetera Investment Advisers trimmed its positio...,Ne,0
2,"Mercer Global Advisors Inc. ADV Takes $312,000...",Mercer Global Advisors Inc. ADV purchased a ne...,Ne,0



## 5. Process All Articles and Save to CSV

Now let's process all articles in a batch and save the results to a CSV file.


In [58]:

# Process all articles
df_processed = process_news_articles(config, page_size=30, days_back=30)

# Display summary
print(f"\nProcessed {len(df_processed)} articles")
print("\nSentiment distribution:")
print(df_processed['sentiment'].value_counts())

# Save to CSV
output_file = config.DATA_DIR / "bitcoin_demo_results.csv"
df_processed.to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

# Show the results
df_processed.head()


Fetching Bitcoin news from 2025-04-17 to 2025-05-17...
Fetched 30 articles
[NEW] [Pos] Refer-a-Friend and You BOTH get a $20 BTC Bonus (Plus BOTH get $5 in Reward Poin...
[CACHED] [N] Cetera Investment Advisers Decreases Holdings in MARA Holdings, Inc. (NASDAQ:MAR...
[CACHED] [N] Mercer Global Advisors Inc. ADV Takes $312,000 Position in Bitwise Bitcoin ETF (...
[CACHED] [N] Bitcoin Could See Short-Term Holder Selloff Around These Levels, Analyst Says...
[CACHED] [N] Monero: Will an 82% rally in 40 days help XMR reach $489?...
[CACHED] [Pos] Bitcoin hitting $220K 'reasonable' in 2025 says gold-based forecast...
[CACHED] [N] Mercer Global Advisors Inc. ADV Makes New $293,000 Investment in CompoSecure, In...
[CACHED] [Pos] Bitcoin Bulls Face $120M Challenge in Extending 'Stair-Step' Uptrend...
[CACHED] [N] Comerica Bank Sells 439 Shares of Grayscale Bitcoin Trust (NYSEARCA:GBTC)...
[CACHED] [Pos] Ukraine Strategic Bitcoin Reserve Bill Reportedly In Final Stages...
[CACHED] [N] Google fix

,publishedAt,headline,sentiment,score,url,cost,cached
0,2025-05-16 08:10:15+00:00,Refer-a-Friend and You BOTH get a $20 BTC Bonu...,Pos,1,https://www.ozbargain.com.au/node/906089,0.0,False
1,2025-05-16 08:07:00+00:00,Cetera Investment Advisers Decreases Holdings ...,N,0,https://www.etfdailynews.com/2025/05/16/cetera...,0.0,True
2,2025-05-16 08:00:53+00:00,"Mercer Global Advisors Inc. ADV Takes $312,000...",N,0,https://www.etfdailynews.com/2025/05/16/mercer...,0.0,True
3,2025-05-16 08:00:49+00:00,Bitcoin Could See Short-Term Holder Selloff Ar...,N,0,http://www.newsbtc.com/bitcoin-news/bitcoin-sh...,0.0,True
4,2025-05-16 08:00:28+00:00,Monero: Will an 82% rally in 40 days help XMR ...,N,0,https://ambcrypto.com/monero-will-an-82-rally-...,0.0,True



## 6. Read Results from CSV

Let's read back the results from the CSV file to demonstrate persistence.


In [59]:

# Read from CSV
df_from_csv = pd.read_csv(output_file)

# Convert date column back to datetime
df_from_csv['publishedAt'] = pd.to_datetime(df_from_csv['publishedAt'])

print(f"Read {len(df_from_csv)} articles from CSV")
df_from_csv.head()


Read 30 articles from CSV


,publishedAt,headline,sentiment,score,url,cost,cached
0,2025-05-16 08:10:15+00:00,Refer-a-Friend and You BOTH get a $20 BTC Bonu...,Pos,1,https://www.ozbargain.com.au/node/906089,0.0,False
1,2025-05-16 08:07:00+00:00,Cetera Investment Advisers Decreases Holdings ...,N,0,https://www.etfdailynews.com/2025/05/16/cetera...,0.0,True
2,2025-05-16 08:00:53+00:00,"Mercer Global Advisors Inc. ADV Takes $312,000...",N,0,https://www.etfdailynews.com/2025/05/16/mercer...,0.0,True
3,2025-05-16 08:00:49+00:00,Bitcoin Could See Short-Term Holder Selloff Ar...,N,0,http://www.newsbtc.com/bitcoin-news/bitcoin-sh...,0.0,True
4,2025-05-16 08:00:28+00:00,Monero: Will an 82% rally in 40 days help XMR ...,N,0,https://ambcrypto.com/monero-will-an-82-rally-...,0.0,True



## 7. Visualize Sentiment Distribution

Finally, let's visualize the sentiment distribution using the helper function.


In [60]:
# Normalize sentiment labels to ensure only three categories
sentiment_map = {
    'Pos': 'Positive',
    'pos': 'Positive',
    'Positive': 'Positive',
    'positive': 'Positive',
    'Neg': 'Negative',
    'neg': 'Negative',
    'Negative': 'Negative',
    'negative': 'Negative',
    'N': 'Neutral',
    'n': 'Neutral',
    'Ne': 'Neutral',
    'ne': 'Neutral',
    'Neutral': 'Neutral',
    'neutral': 'Neutral'
}
df_from_csv['sentiment'] = df_from_csv['sentiment'].map(lambda x: sentiment_map.get(str(x).strip(), 'Neutral'))

In [61]:

# Create pie chart
fig = create_sentiment_pie_chart(df_from_csv)
fig.show()



## Summary

This notebook demonstrated how to:
1. Load configuration from environment variables
2. Fetch Bitcoin news articles
3. Analyze sentiment on individual articles
4. Process multiple articles and save results to CSV
5. Read results back from CSV
6. Visualize sentiment distribution

The caching mechanism ensures that previously analyzed articles are not processed again, saving API tokens.
